# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [2]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

NameError: ignored

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [15]:
import pandas as pd
import random

recipes = pd.read_csv("recipes_sample.csv")

random_ids = random.sample(range(len(recipes)), 5)

print("|{:^12}|{:^10}|".format("id", "minutes"))
print("--------------------")
for i in random_ids:
    print("|{:^12}|{:^10}|".format(recipes.loc[i, "id"], recipes.loc[i, "minutes"]))

|     id     | minutes  |
--------------------
|   120391   |   320    |
|   104098   |    35    |
|   214395   |    90    |
|   255425   |    10    |
|   33508    |    27    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [16]:
def show_info(name, steps, minutes, author_id):
    title = name.title()
    steps_str = "\n".join([f"{i + 1}. {step}" for i, step in enumerate(steps)])
    
    return f'"{title}"\n\n{steps_str}\n----------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n'

print(show_info("george s at the cove black bean soup", ["clean the leeks and discard the dark green portions", "cut the leeks lengthwise then into one-inch pieces", "melt the butter in a medium skillet , med"], 90, 35193))

"George S At The Cove Black Bean Soup"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
----------
Автор: 35193
Среднее время приготовления: 90 минут



In [17]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

AssertionError: ignored

## Работа с регулярными выражениями

In [31]:
import pandas as pd
import numpy as np
import re
from xml.etree import ElementTree
import nltk

# Загрузка данных из файла recipes_sample.csv
recipes = pd.read_csv('recipes_sample.csv', usecols=['id', 'description'])

# Загрузка данных из файла steps_sample.xml
tree = ElementTree.parse('steps_sample.xml')
root = tree.getroot()

# Получение шагов для каждого рецепта
steps_data = []
for recipe in root.findall('recipe'):
    recipe_id = int(recipe.find('id').text)
    steps = ' '.join([step.text for step in recipe.find('steps')])
    steps_data.append((recipe_id, steps))

# Создание DataFrame с шагами
steps_df = pd.DataFrame(steps_data, columns=['id', 'steps'])

# Объединение двух DataFrame
recipes = recipes.merge(steps_df, on='id')

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [32]:
# Регулярное выражение для поиска
pattern = r'\d+\s(?:hour|hours|minute|minutes)'

# Найдем шаги рецепта с id 25082
recipe_25082_steps = recipes.loc[recipes['id'] == 25082, 'steps'].iloc[0]

# Поиск по регулярному выражению
matches = re.findall(pattern, recipe_25082_steps)

# Вывод результатов
print(matches)

['20 minute', '10 minute', '2 hour', '10 minute', '20 minute', '30 minute']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [40]:
# Замена пропущенных значений пустыми строками
recipes['description'].fillna('', inplace=True)

In [41]:
# Регулярное выражение для поиска
pattern = r'^this[^.,;!?]*, ?but'

# Функция для проверки соответствия шаблону
def match_pattern(description):
    return bool(re.match(pattern, description))

# Фильтрация и вывод результатов
matching_recipes = recipes[recipes['description'].apply(match_pattern)]
print(f"Количество рецептов, соответствующих шаблону: {len(matching_recipes)}")
print("Примеры описаний:")
print(matching_recipes['description'].head(3))

Количество рецептов, соответствующих шаблону: 166
Примеры описаний:
76     this is a great meal eaten the same day ,but e...
183    this was adapted from a recipe i found on the ...
337    this is kind of similar to some of the other v...
Name: description, dtype: object


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [34]:
# Регулярное выражение для поиска
pattern = r'\s*/\s*'

# Найдем шаги рецепта с id 72367
recipe_72367_steps = recipes.loc[recipes['id'] == 72367, 'steps'].iloc[0]

# Удаление пробелов вокруг символа дроби
modified_steps = re.sub(pattern, '/', recipe_72367_steps)

# Вывод результатов
print(modified_steps)

mix butter , flour , 1/3 c sugar and 1-1/4 t vanilla press into greased 9" springform pan mix cream cheese , 1/4 c sugar , eggs and 1/2 t vanilla beating until fluffy pour over dough combine apples , 1/3 c sugar and cinnamon arrange on top of cream cheese mixture and sprinkle with almonds bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [35]:
# Разбиение текста на слова
words = nltk.word_tokenize(' '.join(recipes['steps']))

# Фильтрация алфавитных слов и приведение к нижнему регистру
words = [word.lower() for word in words if word.isalpha()]

# Подсчет уникальных слов
unique_words = len(set(words))

# Вывод результатов
print(f"Количество уникальных слов: {unique_words}")

Количество уникальных слов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [39]:
from nltk.tokenize import sent_tokenize

# Разбиение описаний на предложения
recipes['sentences'] = recipes['description'].apply(sent_tokenize)

# Подсчет количества предложений
recipes['num_sentences'] = recipes['sentences'].apply(len)

# Вывод 5 самых длинных описаний
longest_descriptions = recipes.nlargest(5, 'num_sentences')
print(longest_descriptions)

           id                                        description  \
18408  334113  this wonderful icing is used for icing cakes a...   
481    287008  a translucent golden-brown crust allows the gr...   
22566  328708  this is one of the best soups i've ever made a...   
6779   205348  i wrote this because there are an astounding l...   
16296  316000  the first time i made this cake i grated a mil...   

                                                   steps  \
18408  you can purchase meringue powder at your local...   
481    dip: put the crme frache into a medium-sized m...   
22566  in a soup pot on med heat , melt 1 stick of bu...   
6779   place water and mints into pot or sauce pan an...   
16296  sift dry ingredients into a large mixer bowl a...   

                                               sentences  num_sentences  
18408  [this wonderful icing is used for icing cakes ...             76  
481    [a translucent golden-brown crust allows the g...             27  
22566  [

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [37]:
def display_pos(sentence):
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)
    formatted_tags = [tag.ljust(len(word)) for (word, tag) in pos_tags]
    print(' '.join(formatted_tags))
    print(sentence)

# Найдем название рецепта с id 241106
recipe_241106_title = recipes.loc[recipes['id'] == 241106, 'description'].iloc[0]

# Вывод частей речи для названия рецепта
display_pos(recipe_241106_title)

DT    VBP DT RB     JJ   NN    NN   TO VB   IN DT  NN         WRB  JJ    NNS   VBP  JJ    NN       CC  NNS      VBG    IN . DT  VBG     VBN   VBZ DT NN         , CC  EX    VBP RB        JJ   JJR  NNS  TO VB    DT   NN   . VB    IN  IN   DT  JJ       NNS    CC  RB   VB  IN   WP   VBZ    JJ   TO PRP .
these are a really good quick meal to make in the summertime when local farms have fresh eggplant and tomatoes coming in.  the topping given is a suggestion, but there are certainly many more ways to enjoy this dish.  start out with the eggplant steaks and then top with what sounds good to you.
